# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до `0.75`. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем нужные модули и открываем файл с данными:

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import json

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

df = pd.read_csv('...')
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Как видно, четыре первых столбца (`calls`, `minutes`, `messages` и `mb_used`) задают входные признаки, на которых нужно научиться предсказывать последний столбец - тарифный план, который больше подойдёт абоненту, описываемому данной строкой входных признаков. Здесь `is_ultra==1` означает, что такому абоненту нужно рекомендовать тариф "_Ultra_", а значание `is_ultra==0` говорит об уместности рекомендации тарифа "_Smart_". 

Наличие только двух возможных значений целевого признака говорит о том, что мы будем решать задачу бинарной классификации.

Создадим отдельные датафреймы с входными признаками (`features`) и целевым (`target`):

In [16]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

display(features.head())
display(target.to_frame().head())

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


,is_ultra
0,0
1,0
2,0
3,1
4,0


## Разбейте данные на выборки

Используя `sklearn.model_selection.train_test_split()` дважды, разделим наши данные на 3 части: тренировочную (`features_train` и `target_train`), валидационную (`features_valid` и `target_valid`) и тестовую (`features_test` и `target_test`).

При этом сначала выделим `25%` данных для тестовой части. Оставшиеся `75%` разделим ещё раз, выделяя `80%` от них для тренировочной части и `20%` - для валидационной:

In [17]:
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.2, random_state=12345)

print(f'Размеры набора тренировочных входных признаков: {features_train.shape}')
print(f'Размеры набора валидационных входных признаков: {features_valid.shape}')
print(f'Размеры набора тестовых входных признаков: {features_test.shape}')
print('-' * 75)
print(f'Размеры набора тренировочного целевого признака: {target_train.shape}')
print(f'Размеры набора валидационного целевого признака: {target_valid.shape}')
print(f'Размеры набора тестового целевого признака: {target_test.shape}')

Размеры набора тренировочных входных признаков: (1928, 4)
Размеры набора валидационных входных признаков: (482, 4)
Размеры набора тестовых входных признаков: (804, 4)
---------------------------------------------------------------------------
Размеры набора тренировочного целевого признака: (1928,)
Размеры набора валидационного целевого признака: (482,)
Размеры набора тестового целевого признака: (804,)


## Исследуйте модели

Попробуем натренировать три модели на обучающем наборе и оценить их качество (долю правильных ответов, `accuracy`) на валидационном.

Будем использовать следующие три известные мне модели:

- логистическую регрессию (`LogisticRegression`);
- дерево решений (`DecisionTreeClassifier`);
- случайный лес (`RandomForestClassifier`)

Сохраняя качественные показатели каждой из этих моделей, выведем их все вместе, чтобы впоследствии выбрать лучшую:

In [23]:
def record_score(model, models_accuracies):
    """Тренирует модель model на тренировочном наборе данных и сохраняет качество предсказаний
    обученной модели на валидационном наборе в словаре models_accuracies"""
    # Обучаемся на обучающей выборке
    model.fit(features_train, target_train)
    # Делаем предсказания на валидационной выборке
    predictions_valid = model.predict(features_valid)
    # Оцениваем качество предсказаний, сохраняя результат в словарь
    models_accuracies[model.__class__.__name__] = accuracy_score(target_valid, predictions_valid)

models_accuracies = dict()

record_score(LogisticRegression(), models_accuracies)
record_score(DecisionTreeClassifier(), models_accuracies)
record_score(RandomForestClassifier(), models_accuracies)

print(json.dumps(models_accuracies, indent=4))

{
    "LogisticRegression": 0.7365145228215768,
    "DecisionTreeClassifier": 0.7489626556016598,
    "RandomForestClassifier": 0.8112033195020747
}


Можно видеть, что случайный лес (`RandomForestClassifier`) демонстрирует самые высокие показатели `accuracy`. Поэтому, взяв именно эту модель за основу, будем настраивать её гиперпараметры, чтобы добиться по возможности ещё более высокого качества. Настраивать её гиперпараметры будем снова на валидационной выборке, чтобы до последнего "_не показывать_" модели тестовые данные, а использовать их уже только при финальном прогоне с выбранными значениями гиперпараметров.

Чтобы ускорить и как-то автоматизировать процесс перебора возможных значений гиперпараметров, будем использовать рандомизированный поиск среди всевозможных значений, который реализован в `RandomizedSearchCV`. В частности, будем искать лучшее значение среди следующих параметров:

- `n_estimators` - количество деревьев в случайном лесу;
- `max_depth` - максимальная допустимая глубина дерева;
- `min_samples_split` - минимальное количество обучающих примеров для выполнения разбиения в нелистовом узле;
- `min_samples_leaf` - минимальное количество обучающих примеров, которое необходимо, чтобы узел дерева мог бы быть листовым.

Кросс-валидацию будем делать с использованием 15 разбиений (`cv=15`):

In [32]:
param_grid=[
    {
        'n_estimators': [50, 100, 150, 200],
        'max_depth': range(1, 20),
        'min_samples_split': range(1, 20),
        'min_samples_leaf': range(1, 20)
    }]

rs = RandomizedSearchCV(
                 estimator=RandomForestClassifier(random_state=12345),
                 param_distributions=param_grid,
                 scoring='accuracy',
                 cv=15,
                 random_state=12345)

rs.fit(features_train_valid, target_train_valid)
print(f'Максимальное достигнутое качество модели при рандомизированном поиске:\n\t {rs.best_score_}')
print(f'Значения гиперпараметров, обеспечивающие это качество:\n\t {rs.best_params_}')

Максимальное достигнутое качество модели при рандомизированном поиске:
	 0.8203467908902692
Значения гиперпараметров, обеспечивающие это качество:
	 {'n_estimators': 100, 'min_samples_split': 12, 'min_samples_leaf': 3, 'max_depth': 9}


## Проверьте модель на тестовой выборке

Теперь, когда были выбраны наиболее оптимальные значения гиперпараметров, создадим модель случайного леса, передавая эти оптимальные значения в качестве параметров.

Наконец, оценим качество нашей модели теперь уже на тестовой выборке, чтобы проверить удалось ли нам преодолеть допустимое и приемлемое значение доли правильных ответов - `0.75`:

In [30]:
model = RandomForestClassifier(random_state=12345,
                               n_estimators=rs.best_params_['n_estimators'],
                               min_samples_split=rs.best_params_['min_samples_split'],
                               min_samples_leaf=rs.best_params_['min_samples_leaf'],
                               max_depth=rs.best_params_['max_depth'])

model.fit(features_train, target_train)

predictions_test = model.predict(features_test)

test_accuracy = accuracy_score(target_test, predictions_test)
print(f'Качество на тестовой выборке:\n\t {test_accuracy}')

Качество на тестовой выборке
	: 0.8022388059701493


Можно видеть, что качество на тестовой выборке - `0.8` - превосходит приемлемое значение (`0.75`).

## (бонус) Проверьте модели на адекватность

Наконец, попробуем проверить обученную модель на адекватность. Если я правильно понял обсуждение в нашем Slack-канале "`projects`", нам нужно сравнить качество нашей модели с качеством лучшей константной модели - модели, которая для любого входа предсказывает метку наиболее часто встречающегося на обучающей выборке класса.

Начнём с того, что определим, какой класс на обучающем наборе является самым часто встречающимся:

In [34]:
value_counts = target_train.value_counts()
smart_count = value_counts[0]
ultra_count = value_counts[1]
print(f'Количество абонентов тарифа "Smart" (значение метки 0) : {smart_count}')
print(f'Количество абонентов тарифа "Ultra" (значение метки 1) : {ultra_count}')

Количество абонентов тарифа "Smart" (значение метки 0) : 1338
Количество абонентов тарифа "Ultra" (значение метки 1) : 590


Можем видеть, что для нас наиболее часто встречающимся классом является `0` (тариф "_Smart_"). Попробуем оценить, какого качества модели мы могли бы добиться (и на обучающем, и на тестовом наборах), если бы предсказывали наиболее часто встречающийся класс:

In [36]:
dumb_train = np.full((target_train.shape), 0)
dumb_test = np.full((target_test.shape), 0)

print(f'Доля правильных ответов для константной модели:\n')
print(f'\t- на обучающем наборе : {accuracy_score(target_train, dumb_train)}')
print(f'\t- на тестовом наборе  : {accuracy_score(target_test, dumb_test)}')

Доля правильных ответов для константной модели:

	- на обучающем наборе : 0.6939834024896265
	- на тестовом наборе  : 0.7002487562189055


Можно видеть, что качество константной модели составляет около `0.7`. Исходя из этого, можно более ясно понять выбор приемлемого значения в `0.75` - это примерно на `5%` лучше совсем тривиальной модели, которая всегда предсказывает одно и то же значение.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75